<a href="https://colab.research.google.com/github/oonid/growth-hacking-with-nlp-sentiment-analysis/blob/master/create_neural_network_based_sentiment_analyzers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network Based Sentiment Analyzers

### requirements setup

includes: 

*   transformers
*   simpletransformers
*   tokenizers



In [1]:
!wget https://raw.githubusercontent.com/crow-intelligence/growth-hacking-sentiment/master/requirements.txt
!pip install -r requirements.txt

--2020-06-20 18:54:44--  https://raw.githubusercontent.com/crow-intelligence/growth-hacking-sentiment/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     151  --.-KB/s    in 0s      

2020-06-20 18:54:44 (10.4 MB/s) - ‘requirements.txt’ saved [151/151]

     |████████████████████████████████| 194kB 9.2MB/s 
     |████████████████████████████████| 3.8MB 15.5MB/s 
     |████████████████████████████████| 675kB 54.8MB/s 
     |████████████████████████████████| 204kB 48.5MB/s 
     |████████████████████████████████| 890kB 51.7MB/s 
     |████████████████████████████████| 1.1MB 55.1MB/s 
  Created wheel for Keyness: filename=Keyness-0.25-cp36-none-any.

### Install apex

make sure to switch Google Colab runtime type to GPU hardware accelerator


In [2]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [3]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 7274 (delta 9), reused 6 (delta 0), pack-reused 7255
Receiving objects: 100% (7274/7274), 13.87 MiB | 15.24 MiB/s, done.
Resolving deltas: 100% (4908/4908), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-wn285_e6
Created temporary directory: /tmp/pip-req-tracker-y9nvk_eq
Created requirements tracker '/tmp/pip-req-tracker-y9nvk_eq'
Created temporary directory: /tmp/pip-install-1arprxgm
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-di2zkidr
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-y9nvk_eq'
    Running setup.py (path:/t

In [4]:
# connect google colab to google driver if needed (uncomment 2 lines below)
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [5]:
# all imports and its related

%matplotlib inline

import pandas as pd
import numpy as np

from transformers import pipeline, BertModel, BertTokenizer


### load the small_corpus CSV

run process from [create_dataset.ipynb](https://github.com/oonid/growth-hacking-with-nlp-sentiment-analysis/blob/master/create_dataset.ipynb)

copy file **small_corpus.csv** to this Google Colab Files (via file upload or mount drive)


In [6]:
df = pd.read_csv('small_corpus.csv')
df

,ratings,reviews
0,1,Recently UBISOFT had to settle a huge class-ac...
1,1,"code didn't work, got me a refund."
2,1,"these do not work at all, all i get is static ..."
3,1,well let me start by saying that when i first ...
4,1,"Dont waste your money, you will just end up us..."
...,...,...
4495,5,"Nice long micro USB cable, battery lasts a lon..."
4496,5,I've been having a great time with this game. ...
4497,5,d
4498,5,"Really pretty, funny, interesting game. Works ..."


In [7]:
# check if any columns has null, and yes the reviews column has
df.isnull().any()

ratings    False
reviews     True
dtype: bool

In [8]:
# repair null in column reviews with empty string ''
df.reviews = df.reviews.fillna('')

# test again
df.isnull().any()

ratings    False
reviews    False
dtype: bool

In [9]:
# Setting up transformers pipeline and tokenizer
sentiment_analysis_pipeline = pipeline('sentiment-analysis')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


In [10]:
for row in pd.concat([df.head(2), df.tail(2)]).itertuples(index=False):
    # print(row.reviews)
    review = row.reviews
    if len(review) > 512: # chop to 512, maximum sequence length for this model
        review = review[:512]
        print('[warning] review get chopped.')
        # TODO find alternative to handle long sentences,
        # TODO shall we tokenize to multiple sentences and process per sentence?
    e = tokenizer.encode(review)
    d = tokenizer.decode(e, skip_special_tokens=True)
    print(d)
    s = sentiment_analysis_pipeline(d)
    print(s)
    print('\n--\n')


[warning] review get chopped.
recently ubisoft had to settle a huge class - action suit brought against the company for bundling ( the notoriously harmful ) starforce drm with its released games. so what the geniuses at the helm do next? they decide to make the same mistake yet again - by choosing the same drm scheme that made bioshock, mass effect and spore infamous : securom 7. xx with limited activations! mass effect can be found in clearance bins only months after its release ; spore not only undersold miserably but also made history as t
[{'label': 'NEGATIVE', 'score': 0.9991271495819092}]

--

code didn't work, got me a refund.
[{'label': 'NEGATIVE', 'score': 0.9996480345726013}]

--

really pretty, funny, interesting game. works well. i recommend it for all ages.
[{'label': 'POSITIVE', 'score': 0.9998774528503418}]

--

i had a lot of fun playing this game, if your looking for a game to jump into with not much overthinking this is the game
[{'label': 'POSITIVE', 'score': 0.98784

In [11]:
def score_review(review):
    if len(review) > 512: # chop to 512, maximum sequence length for this model
        # print('\n[warning] review get chopped.\n{}'.format(review))
        review = review[:512]
        # TODO find alternative to handle long sentences,
        # TODO shall we tokenize to multiple sentences and process per sentence?
    e = tokenizer.encode(review)
    d = tokenizer.decode(e, skip_special_tokens=True)
    s = sentiment_analysis_pipeline(d)
    if len(s) != 1:
        raise ValueError('got multiple sentiment analysis results.')

    # s[0] the first and only sentiment analysis result
    if s[0]['label'] == 'POSITIVE' and s[0]['score'] > 0.85:
        return 1
    elif s[0]['label'] == 'NEGATIVE' and s[0]['score'] > 0.95:
        return -1
    else:  # NEUTRAL
        return 0

sentiment_classes = df['reviews'].apply(score_review)
sentiment_classes

0      -1
1      -1
2      -1
3      -1
4      -1
       ..
4495   -1
4496    1
4497    1
4498    1
4499    1
Name: reviews, Length: 4500, dtype: int64

In [12]:
def categorize_rating(rating):
    if rating == 5:
        return 1  # 'positive'
    elif 2 <= rating <= 4:
        return 0  # 'neutral'
    else:  # rating == 1
        return -1  # 'negative'

rating_classes = df['ratings'].apply(categorize_rating)
rating_classes

0      -1
1      -1
2      -1
3      -1
4      -1
       ..
4495    1
4496    1
4497    1
4498    1
4499    1
Name: ratings, Length: 4500, dtype: int64

In [13]:
df['sentiment classes'] = sentiment_classes
df['rating classes'] = rating_classes
df

,ratings,reviews,sentiment classes,rating classes
0,1,Recently UBISOFT had to settle a huge class-ac...,-1,-1
1,1,"code didn't work, got me a refund.",-1,-1
2,1,"these do not work at all, all i get is static ...",-1,-1
3,1,well let me start by saying that when i first ...,-1,-1
4,1,"Dont waste your money, you will just end up us...",-1,-1
...,...,...,...,...
4495,5,"Nice long micro USB cable, battery lasts a lon...",-1,1
4496,5,I've been having a great time with this game. ...,1,1
4497,5,d,1,1
4498,5,"Really pretty, funny, interesting game. Works ...",1,1


In [14]:
from sklearn.metrics import accuracy_score

y_pred = list(df['sentiment classes'])
y_true = list(df['rating classes'])

print('accuracy score: {}'.format(accuracy_score(y_true=y_true, y_pred=y_pred)))
print('accuracy score (normalized): {}'\
      .format(accuracy_score(y_true=y_true, y_pred=y_pred, normalize=True)))

accuracy score: 0.6106666666666667
accuracy score (normalized): 0.6106666666666667


In [15]:
from sklearn.metrics import classification_report

target_names = ['class Negative', 'class Neutral', 'class Positive']

print('classification report:\n{}'\
      .format(classification_report(y_true=y_true, y_pred=y_pred,
                                    target_names=target_names)))

classification report:
                precision    recall  f1-score   support

class Negative       0.59      0.89      0.71      1500
 class Neutral       0.43      0.08      0.14      1500
class Positive       0.67      0.86      0.75      1500

      accuracy                           0.61      4500
     macro avg       0.56      0.61      0.53      4500
  weighted avg       0.56      0.61      0.53      4500

